<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/%E3%82%B8%E3%83%A3%E3%83%91%E3%83%B3%E3%82%B5%E3%83%BC%E3%83%81%E3%81%AERDF%E3%82%B9%E3%83%88%E3%82%A2%E3%82%92%E5%AF%BE%E8%B1%A1%E3%81%97%E3%81%9FSPARQL%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ジャパンサーチのRDFストアを対象したSPARQLチュートリアル

ジャパンサーチのRDFストアを対象したSPARQLのチュートリアルです。

参考：https://jpsearch.go.jp/static/developer/ja.html

## 初期設定

In [ ]:
# 日本語の文字化け対策
!pip install japanize-matplotlib
import requests
import urllib
import json
import pprint
import matplotlib.pyplot as plt
import japanize_matplotlib

endpoint = "https://jpsearch.go.jp/rdf/sparql/"

def execQuery(q):
  url = "{}?query={}&format=json&output=json&results=json".format(endpoint, urllib.parse.quote(q))
  return requests.get(url).json()

def displayGraph(X, Y, title, xlabel, ylabel):
  # グラフの大きさ指定
  plt.figure(figsize=(10, 10))

  # 棒グラフを表示
  plt.barh(X, Y)

  # XとYのラベル
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)

  # タイトル表示
  plt.title(title, fontsize=15)

## 例1:作品数の多い順に作者を取得する

### クエリ例

In [ ]:
q = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX chname: <https://jpsearch.go.jp/entity/chname/>
SELECT distinct (count(?cho) as ?count) ?label WHERE {
    ?cho schema:creator ?creator .
    ?creator rdfs:label ?label . 
    ?creator rdfs:isDefinedBy chname: # 正規化されている名前のみ

    # 以下のように書くことができます。

    # ?cho schema:creator ?creator .
    # ?creator rdfs:label ?label; rdfs:isDefinedBy chname: # 正規化されている名前のみ
}
ORDER BY DESC(?count)
LIMIT 20
"""

results = execQuery(q)
for obj in results["results"]["bindings"]:
  # pprint.pprint(obj)
  count = obj["count"]["value"]
  label = obj["label"]["value"]
  print(label, "\t", count)

### 可視化例

In [ ]:
x_list = []
x_value = "label"
y_list = []
y_value = "count"
for obj in results["results"]["bindings"]:
  x_list.append(obj[x_value]["value"])  
  y_list.append(int(obj[y_value]["value"]))

displayGraph(x_list, y_list, "作品数の多い作者", "作品数", "作者", )  